In [1]:
from vod.configuration import KittiLocations
from vod.frame import FrameTransformMatrix, transform_pcl, FrameDataLoader
import numpy as np
import k3d
from my_fuse_radar.utils import read_kitti_label_file, plot_moving_objs_bbox
from vod.visualization import get_radar_velocity_vectors
kitti_locations = KittiLocations(root_dir='/datasets/vod')

In [2]:
frame_str = '00544'
frame_data = FrameDataLoader(kitti_locations=kitti_locations, frame_number=frame_str)
transforms = FrameTransformMatrix(frame_data)
_, moving_objs = read_kitti_label_file(frame_data.raw_labels)
lidar_points_cf = transform_pcl(frame_data.lidar_data, transforms.t_camera_lidar)[:,:3]
radar_points_cf = transform_pcl(frame_data.radar_data, transforms.t_camera_radar)[:,:3] # (N, 3)

label_mos = np.fromfile('/datasets/vod/my_radar_5frames/training/label_mos/00544.label', dtype=np.uint8)
print(len(label_mos))
print(label_mos)
moving_points = np.array(radar_points_cf[label_mos == 1])
static_points = np.array(radar_points_cf[label_mos == 0])

print(f'Number of radar points: {radar_points_cf.shape[0]}')
print(f'Number of moving points: {moving_points.shape[0]}')
print(f'Number of static points: {static_points.shape[0]}')

radial_velocity = frame_data.radar_data[:, 5]
points_velo_vectors = get_radar_velocity_vectors(radar_points_cf, radial_velocity)

326
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1
 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Number of radar points: 326
Number of moving points: 32
Number of static points: 294


In [3]:
plot = k3d.plot(camera_auto_fit=False, axes_helper=True, grid_visible=False)
plot += k3d.points(np.array([0,0,0]), point_size=0.5, color=0x00FF00) # 原点
plot += k3d.points(lidar_points_cf, point_size=0.05, color=0x000000)
plot += k3d.points(radar_points_cf, point_size=0.05, color=0xFF0F00)
plot_moving_objs_bbox(plot, moving_objs, transforms.t_lidar_camera, transforms.t_camera_lidar)

plot += k3d.points(moving_points, point_size=0.3, color=0xFF0000)
plot += k3d.points(static_points, point_size=0.3, color=0x0000FF)
plot += k3d.vectors(origins=radar_points_cf, vectors=points_velo_vectors, color=0xFF0000)
plot.display()

/home/yu/miniconda3/envs/openpcdet/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/yu/miniconda3/envs/openpcdet/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()